# Reverse Geocoding Citi Bike Stations

[Credit](https://towardsdatascience.com/reverse-geocoding-with-nyc-bike-share-data-cdef427987f8/)

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/citibike-stations.csv', index_col=[0])
df.reset_index(drop=True, inplace=True)
df.head()

,station_name,station_id,latitude,longitude
0,W 36 St & 7 Ave,6483.06,40.752149,-73.989539
1,Broadway & E 19 St,5980.11,40.738290,-73.990060
2,Jackson Ave & 46 Rd,6203.02,40.745248,-73.947333
3,Willis Ave & E 143 St,7798.02,40.812299,-73.920370
4,Grand Concourse & E 167 St,8126.07,40.834263,-73.917641


In [3]:
geo = Nominatim(user_agent="kazirshahria@gmail.com")
reverse = RateLimiter(geo.reverse, min_delay_seconds=1, max_retries=2)

In [4]:
def reverse_geolocation(lat, long):
    coordinate = f'{lat}, {long}'
    response = reverse(coordinate)
    address = response.raw['address']
    neighborhood = address.get('neighbourhood')
    borough = address.get('borough')
    county = address.get('county')
    city = address.get('city')
    state = address.get('state')
    postcode = address.get('postcode')
    return (neighborhood, borough, county, city, state, postcode)

values = df.apply(lambda col: reverse_geolocation(col['latitude'], col['longitude']), axis=1)

In [5]:
df[['neighborhood', 'borough', 'county', 'city', 'state', 'postcode']] = values.apply(pd.Series)
df.head()

,station_name,station_id,latitude,longitude,neighborhood,borough,county,city,state,postcode
0,W 36 St & 7 Ave,6483.06,40.752149,-73.989539,Garment District,Manhattan,New York County,City of New York,New York,10018
1,Broadway & E 19 St,5980.11,40.738290,-73.990060,Union Square,Manhattan,New York County,City of New York,New York,10003
2,Jackson Ave & 46 Rd,6203.02,40.745248,-73.947333,None,Queens,Queens County,City of New York,New York,11101
3,Willis Ave & E 143 St,7798.02,40.812299,-73.920370,None,The Bronx,Bronx County,City of New York,New York,10454
4,Grand Concourse & E 167 St,8126.07,40.834263,-73.917641,None,The Bronx,Bronx County,City of New York,New York,10456


In [11]:
df.drop_duplicates(subset='station_id', keep='first', inplace=True)

In [12]:
df.to_csv('../data/citibike-stations-clean.csv')

In [13]:
df.to_parquet('../data/citibike-stations.parquet')